# AR6 R10 Scenario Analysis Notebook

## Overview
This notebook processes IPCC AR6 Working Group III scenario data to extract key energy system drivers for VerveStacks energy modeling. It focuses on Chapter 3 vetted scenarios across 5 climate categories (C1, C2, C3, C4, C7) and 10 global regions.

## Data Sources
- **AR6 Scenarios Database R10 regions v1.1**: Main scenario data with 10-region aggregation
- **AR6 Metadata with Climate Indicators**: Chapter 3 vetted scenarios with climate categories
- **Source Directory**: `../data/ipcc_iamc/AR6_R10_v1.1/`

## Key Processing Steps

### 1. Data Loading & Filtering (Cell 1)
- Loads raw AR6 scenarios and vetted metadata
- Filters scenarios to only include Chapter 3 vetted model-scenario combinations
- Excludes R10ROWO region from analysis
- **Output**: `scenarios_df` with vetted scenarios only

### 2. Surgical Outlier Removal (Cell 1-2)
- Applies conservative IQR-based outlier detection (2.5×IQR threshold)
- Removes outliers surgically by Variable-Category-Region groups using 2050 data
- Preserves scenario-model combinations in other variable groups
- **Output**: `scenarios_df_clean` and cleaned CSV file

### 3. Variable Exploration Utilities (Cells 3-4)
- `find_variables()`: Pattern-based variable search with wildcard support
- Enables exploration of AR6 variable taxonomy

### 4. Data Preparation Functions (Cell 5)
- `prepare_one_variable_long()`: Converts wide format to long format for analysis
- `prepare_idx_variable()`: Creates indexed data (relative to base year)
- `prepare_ratio_variable()`: Calculates ratios between variables

### 5. Multi-Variable Analysis (Cells 6-7)
Processes 10 key energy system drivers:

#### Price Variables
- **CO2 Price**: `Price|Carbon` (absolute values)
- **Gas Price**: `Price|Secondary Energy|Gases|Natural Gas` (indexed to 2020)
- **Oil Price**: `Price|Secondary Energy|Liquids|Oil` (indexed to 2020)
- **Coal Price**: `Price|Secondary Energy|Solids|Coal` (indexed to 2020)
- **Biomass Price**: `Price|Secondary Energy|Solids|Biomass` (indexed to 2020)

#### Electrification Variables
- **Electricity Growth**: `Final Energy|Electricity` (indexed to 2020)
- **Hydrogen Share**: `Final Energy|Hydrogen` / `Final Energy|Electricity`
- **Industry Electrification**: `Final Energy|Industry|Electricity` / `Final Energy|Electricity`
- **Residential & Commercial**: `Final Energy|Residential and Commercial|Electricity` / `Final Energy|Electricity`
- **Transportation Electrification**: `Final Energy|Transportation|Electricity` / `Final Energy|Electricity`

### 6. Statistical Analysis
For each variable, calculates by Category-Region-Year:
- Count, mean, median, standard deviation
- Min, max, Q25, Q75 quantiles
- Climate category descriptions

### 7. Visualization (Cell 8)
Creates three CO2 price visualization options:
- **Interactive Trajectory Plot**: Median trends with uncertainty bands
- **Ridge Plots**: Distribution evolution over time
- **Heatmap**: Pattern recognition across categories and years

## Output Files
- `AR6_Scenarios_Database_R10_regions_v1.1_clean.csv`: Cleaned scenario data
- `ar6_r10_scenario_drivers.csv`: Statistical summaries (cleaned data)
- `ar6_r10_scenario_drivers_raw.csv`: Statistical summaries (raw data)

## Climate Categories
- **C1**: Limit warming to 1.5°C (>50%) with no or limited overshoot
- **C2**: Limit warming to 1.5°C (>67%) with high overshoot  
- **C3**: Limit warming to 2°C (>67%) with higher action post-2030
- **C4**: Limit warming to 2°C (>50%) with immediate action
- **C7**: Likely above 3°C warming with limited mitigation

## Usage Notes
- Run cells sequentially for proper variable dependencies
- Cell 6 uses cleaned data, Cell 7 uses raw data for comparison
- Visualization functions are modular and can be adapted for other variables
- All analysis focuses on years 2020-2050 in 5-year intervals


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pickle

print("🔄 Loading AR6 R10 data independently...")

# Load raw Excel files directly (like IEA notebook)
data_dir = Path("../data/ipcc_iamc/AR6_R10_v1.1")

# Read the vetted metadata (Chapter 3 vetted scenarios)
print("📊 Loading AR6 vetted metadata...")
metadata_df = pd.read_excel(data_dir / "AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", 
                           sheet_name='meta_Ch3vetted_withclimate')

print("📈 Loading AR6 scenarios data...")  
scenarios_df = pd.read_csv(data_dir / "AR6_Scenarios_Database_R10_regions_v1.1.csv")

print(f"✅ Raw data loaded:")
print(f"   📋 Vetted metadata: {metadata_df.shape}")
print(f"   📊 Full scenarios: {scenarios_df.shape}")

# Get the vetted model-scenario combinations
print("\n🔍 Filtering for vetted scenarios...")
vetted_combinations = metadata_df[['Model', 'Scenario']].drop_duplicates()
print(f"   ✅ Found {len(vetted_combinations)} unique vetted Model-Scenario pairs")

# Filter scenarios data to only include vetted combinations
scenarios_df = scenarios_df.merge(
    vetted_combinations, 
    on=['Model', 'Scenario'], 
    how='inner'
)

print(f"   📊 Filtered scenarios data: {scenarios_df.shape}")
print(f"   🗑️ Reduction: {len(pd.read_csv(data_dir / 'AR6_Scenarios_Database_R10_regions_v1.1.csv')) - len(scenarios_df):,} rows removed")
print(f"   🎭 Available categories: {sorted(metadata_df['Category'].unique())}")

display(scenarios_df.head())


In [ ]:
def remove_outliers_iqr_surgical(df, outlier_year='2050'):
    """
    Surgically remove outlier scenario-model combinations from specific Variable-Category-Region groups only
    
    Strategy: For each Variable-Category-Region combination:
    1. Apply IQR to 2050 values within that group
    2. Remove outlier scenario-model combinations ONLY from that specific group
    3. Keep those scenarios in other Variable-Category-Region combinations
    
    Parameters:
    - df: DataFrame with data
    - outlier_year: Year to use for outlier detection (default: '2050')
    
    Returns:
    - cleaned_df: DataFrame with surgical outlier removal
    - outlier_summary: Summary of outliers removed by group
    """
    
    print(f"🧹 Applying SURGICAL IQR-based outlier removal using {outlier_year} data...")
    print(f"   🎯 Strategy: Remove scenario-model combinations ONLY from specific Variable-Category-Region groups")
    
    if outlier_year not in df.columns:
        print(f"   ❌ Year {outlier_year} not found in data")
        return df.copy(), pd.DataFrame()
    
         # Key variables and categories for outlier detection
    key_variables = [
        'Price|Carbon',
        'Final Energy|Electricity', 
        'Final Energy|Hydrogen',
        'Final Energy|Industry|Electricity',
        'Final Energy|Residential and Commercial|Electricity',
        'Final Energy|Transportation|Electricity',
        'Price|Secondary Energy|Gases|Natural Gas',
        'Price|Secondary Energy|Liquids|Oil',
        'Price|Secondary Energy|Solids|Biomass',
        'Price|Secondary Energy|Solids|Coal',
    ]
    
    # Focus on the 5 climate categories we care about
    target_categories = ['C1', 'C2', 'C3', 'C4', 'C7']
    
    print(f"   🎯 Processing {len(key_variables)} key variables")
    print(f"   🎭 Target categories: {target_categories}")
    
    # Start with original data
    cleaned_df = df.copy()
    removal_stats = []
    total_removals = 0
    
    # Merge with metadata to get categories
    df_with_category = df.merge(
        metadata_df[['Model', 'Scenario', 'Category']], 
        on=['Model', 'Scenario'], 
        how='left'
    )
    df_with_category = df_with_category.dropna(subset=['Category'])
    
    # Filter to target categories and exclude R10ROWO
    df_with_category = df_with_category[
        (df_with_category['Category'].isin(target_categories)) &
        (df_with_category['Region'] != 'R10ROWO')
    ]
    
    print(f"   🎭 Available categories: {sorted(df_with_category['Category'].unique())}")
    print(f"   🌍 Available regions: {sorted(df_with_category['Region'].unique())}")
    print(f"   🚫 Excluded: R10ROWO region")
    
    # Loop over each Variable-Category-Region combination
    for variable in key_variables:
        var_data = df_with_category[df_with_category['Variable'] == variable].copy()
        
        if var_data.empty:
            print(f"   ⚠️  No data for variable: {variable}")
            continue
            
        print(f"\n   🔍 Processing variable: {variable}")
        
        for category in var_data['Category'].unique():
            for region in var_data['Region'].unique():
                
                # Get specific group data
                group_data = var_data[
                    (var_data['Category'] == category) & 
                    (var_data['Region'] == region) &
                    (var_data[outlier_year].notna())
                ].copy()
                
                if len(group_data) < 4:  # Need at least 4 points for meaningful IQR
                    continue
                
                # Apply IQR to this specific group
                Q1 = group_data[outlier_year].quantile(0.25)
                Q3 = group_data[outlier_year].quantile(0.75)
                IQR = Q3 - Q1
                
                if IQR == 0:  # All values are the same
                    continue
                
                                 # Conservative outlier detection (2.5×IQR)
                lower_bound = Q1 - 2.5 * IQR
                upper_bound = Q3 + 2.5 * IQR
                 
                 # Identify outliers in this group
                outliers = (group_data[outlier_year] < lower_bound) | (group_data[outlier_year] > upper_bound)
                outlier_scenarios = group_data[outliers][['Model', 'Scenario']]
                 
                if len(outlier_scenarios) > 0:
                     # Remove these scenario-model combinations ONLY from this Variable-Category-Region group
                     outlier_combinations = set(zip(outlier_scenarios['Model'], outlier_scenarios['Scenario']))
                     
                     # 🎭 ENTERTAINMENT: Print the actual outlier combinations being removed
                     print(f"      🎯 {variable} | {category} | {region}")
                     print(f"         📊 Group size: {len(group_data)} scenarios")
                     print(f"         📈 2050 range: {group_data[outlier_year].min():.1f} - {group_data[outlier_year].max():.1f}")
                     print(f"         🗑️  Removing {len(outlier_scenarios)} outlier scenarios:")
                     
                     for idx, (model, scenario) in enumerate(outlier_combinations):
                         outlier_value = group_data[
                             (group_data['Model'] == model) & 
                             (group_data['Scenario'] == scenario)
                         ][outlier_year].iloc[0]
                         print(f"            {idx+1}. {model} | {scenario} (2050: {outlier_value:.1f})")
                     
                     # Create mask for this specific group
                     group_mask = (
                         (cleaned_df['Variable'] == variable) & 
                         (cleaned_df.merge(metadata_df[['Model', 'Scenario', 'Category']], on=['Model', 'Scenario'], how='left')['Category'] == category) &
                         (cleaned_df['Region'] == region)
                     )
                     
                     # Remove outlier combinations from this group only
                     scenario_mask = cleaned_df.apply(
                         lambda row: (row['Model'], row['Scenario']) in outlier_combinations, axis=1
                     )
                     
                     removal_mask = group_mask & scenario_mask
                     removed_count = removal_mask.sum()
                     
                     if removed_count > 0:
                         cleaned_df = cleaned_df[~removal_mask].copy()
                         total_removals += removed_count
                         
                         removal_stats.append({
                             'Variable': variable,
                             'Category': category, 
                             'Region': region,
                             'Group_Size': len(group_data),
                             'Outliers_Found': len(outlier_scenarios),
                             'Rows_Removed': removed_count
                         })
                         
                         print(f"         ✅ Removed {removed_count} rows from this specific group")
                         print()
    
    # Create summary
    if removal_stats:
        outlier_summary = pd.DataFrame(removal_stats)
        print(f"\n📊 Surgical Removal Summary:")
        print(f"   🎯 Groups processed: {len(removal_stats)}")
        print(f"   🗑️  Total rows removed: {total_removals:,}")
        print(f"   📈 Original dataset: {len(df):,} rows")
        print(f"   ✨ Cleaned dataset: {len(cleaned_df):,} rows")
        print(f"   📉 Removal rate: {total_removals/len(df)*100:.2f}%")
    else:
        outlier_summary = pd.DataFrame()
        print(f"\n✅ No outliers found in any Variable-Category-Region groups")
    
    return cleaned_df, outlier_summary

# Apply IMPROVED IQR outlier removal using 2050 data only
print(f"🎯 Applying IMPROVED outlier removal based on 2050 data only")
print(f"   🔧 Changes: 2.5×IQR threshold (was 1.5×), key variables only")

scenarios_df_clean, outlier_summary = remove_outliers_iqr_surgical(scenarios_df, outlier_year='2050')

if not outlier_summary.empty:
    print("\n📋 Detailed outlier removal by year:")
    display(outlier_summary)
    
    # Show key statistics
    if len(outlier_summary) > 0:
        avg_removal = outlier_summary['removed_pct'].mean()
        print(f"\n📊 Summary Statistics:")
        print(f"   Average removal rate: {avg_removal:.1f}%")
        print(f"   Years with data: {len(outlier_summary[outlier_summary['original'] > 0])}")
        print(f"   Years with >50% removal: {len(outlier_summary[outlier_summary['removed_pct'] > 50])}")
else:
    print("\n✅ No outliers detected in 2050 data")

In [ ]:
import pandas as pd

scenarios_df_clean.to_csv((data_dir / "AR6_Scenarios_Database_R10_regions_v1.1_clean.csv"), index=False)
outlier_summary.to_csv((data_dir / "AR6_Scenarios_Database_R10_regions_v1.1_outlier_summary.csv"), index=False)

In [ ]:
import re

def find_variables(scenarios_df, pattern):
    """Find variables matching a pattern in the scenarios dataset"""
    variables = scenarios_df['Variable'].unique()
        
    # Convert wildcard pattern to regex
    regex_pattern = '^' + re.escape(pattern).replace(r'\*', '.*').replace(r'\?', '.') + '$'
    regex = re.compile(regex_pattern, re.IGNORECASE)
    matching_vars = [v for v in variables if regex.match(v)]

    return sorted(matching_vars)

# Test the variable search function
print("🔍 Testing variable search...")
results = find_variables(scenarios_df, 'Price|sec*')
print(f"Found {len(results)} variables matching 'Final Energy|Transportation*Electricity':")
for var in results[:15]:  # Show first 5
    print(f"  - {var}")
if len(results) > 15:
    print(f"  ... and {len(results) - 15} more")


In [ ]:
# 🔍 Variable Exploration Utility
def find_variables(scenarios_df, pattern):
    """Find variables matching a pattern in the scenarios dataset"""
    variables = scenarios_df['Variable'].unique()
        
    # Convert wildcard pattern to regex
    regex_pattern = '^' + re.escape(pattern).replace(r'\*', '.*').replace(r'\?', '.') + '$'
    regex = re.compile(regex_pattern, re.IGNORECASE)
    matching_vars = [v for v in variables if regex.match(v)]

    return sorted(matching_vars)

# Quick test - explore transportation electricity variables
results = find_variables(scenarios_df_clean, 'Final Energy|Transportation*Electricity')
print(f"🚗 Found {len(results)} transportation electricity variables:")
for var in results:
    print(f"  - {var}")

print(f"\n💡 Use find_variables(scenarios_df_clean, 'pattern*') to explore any variable!")
print(f"   Examples:")
print(f"   - find_variables(scenarios_df_clean, 'Price|Carbon')")  
print(f"   - find_variables(scenarios_df_clean, '*Hydrogen*')")
print(f"   - find_variables(scenarios_df_clean, 'Final Energy|*')")


In [ ]:
import pandas as pd
import re

"""
Price|Carbon
Final Energy|Electricity
Final Energy|Industry|Electricity
Final Energy|Residential and Commercial|Electricity
Final Energy|Transportation|Electricity
Final Energy|Hydrogen

'Final Energy|Transportation|Freight|Electricity',
 'Final Energy|Transportation|Passenger|Electricity'
"""

def prepare_one_variable_long(scenarios_df_clean, variable_name):
    """Prepare data for distribution analysis"""
        
    # Filter scenarios data for variable == variable_name, case insensitive
    var_data = scenarios_df_clean[scenarios_df_clean['Variable'].str.lower() == variable_name.lower()].copy()
    

    # Identify columns that look like years (e.g., '2000', '2010', etc.)
    year_cols = [col for col in var_data.columns if re.match(r'^\d{4}$', str(col))]

    # Melt the data to long format
    id_cols = ['Model', 'Scenario', 'Region', 'Variable', 'Unit']
    melted_data = var_data.melt(
        id_vars=id_cols,
        value_vars=year_cols,
        var_name='Year',
        value_name='Value'
    )
    
    # Convert Year to integer and filter out NaN values
    melted_data['Year'] = melted_data['Year'].astype(int)
    melted_data = melted_data.dropna(subset=['Value'])
    
    # Merge with metadata to get climate categories
    analysis_data = melted_data.merge(
        metadata_df[['Model', 'Scenario', 'Category', 'Category_name']], 
        on=['Model', 'Scenario'], 
        how='left'
    )
    
    # Remove rows without category information
    analysis_data = analysis_data.dropna(subset=['Category'])
    
    # Filter for available categories only
    selected_categories = ['C1', 'C2', 'C3', 'C4', 'C7']
    analysis_data = analysis_data[analysis_data['Category'].isin(selected_categories)]

    selected_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]
    analysis_data = analysis_data[analysis_data['Year'].isin(selected_years)]


    return analysis_data

def prepare_idx_variable(variable_name, idx_year):
    """Prepare data for index analysis"""

    # Divide all matching values of a df by year = idx_year. value field = Value
    # Assumes var_df has columns: ['Model', 'Scenario', 'Region', 'Variable', 'Unit', 'Year', 'Value', ...]
    # For each group (Model, Scenario, Region, Variable, Unit, ...), divide Value by the Value at idx_year

    var_df = prepare_one_variable_long(scenarios_df_clean, variable_name)

    # Identify grouping columns (all except 'Year' and 'Value')
    group_cols = [col for col in var_df.columns if col not in ['Year', 'Value']]

    # Get the reference values at idx_year
    ref_df = var_df[var_df['Year'] == idx_year][group_cols + ['Value']].rename(columns={'Value': 'Value_ref'})

    # Merge reference values back to the full dataframe
    merged = var_df.merge(ref_df, on=group_cols, how='left')

    # Divide Value by Value_ref
    merged['Value'] = merged['Value'] / merged['Value_ref']

    # Drop the Value_ref column
    merged = merged.drop(columns=['Value_ref'])

    return merged

def prepare_ratio_variable(variable_num, variable_denom):
    """Prepare data for ratio analysis"""

    var_df_num = prepare_one_variable_long(scenarios_df_clean, variable_num)

    var_df_denom = prepare_one_variable_long(scenarios_df_clean, variable_denom)



    # Merge the numerator and denominator dataframes on common columns
    group_cols = [col for col in var_df_num.columns if col not in ['Value', 'Variable']]

    merged = var_df_num.merge(var_df_denom, on=group_cols, how='inner',suffixes=('_num', '_denom'))  

    # Divide Value_num by Value_denom
    merged['Value'] = merged['Value_num'] / merged['Value_denom']

    # Drop the Value_num and Value_denom columns
    merged = merged.drop(columns=['Value_num', 'Value_denom'])

    return merged



# idx_df = prepare_idx_variable('Final Energy|Electricity', 2020)


# display(idx_df)

ratio_df = prepare_ratio_variable('Final Energy|Hydrogen','Final Energy|Electricity')

display(ratio_df)

In [ ]:
# with clean data - outliers removed

import pandas as pd

def analyze_one_variable(var_data,var_name):
    """Analyze distribution of a variable across scenarios and regions"""
    
    category_descriptions = {
        'C1': 'Limit warming to 1.5°C (>50%) with no or limited overshoot',
        'C2': 'Limit warming to 1.5°C (>67%) with high overshoot',
        'C3': 'Limit warming to 2°C (>67%) with higher action post-2030', 
        'C4': 'Limit warming to 2°C (>50%) with immediate action',
        'C7': 'Likely above 3°C warming with limited mitigation'
    }
    
    # Calculate summary statistics for each category-region-year combination
    summary_stats = var_data.groupby(['Category', 'Region', 'Year'])['Value'].agg([
        'count', 'mean', 'median', 'std', 'min', 'max',
        lambda x: x.quantile(0.25),  # Q1
        lambda x: x.quantile(0.75)   # Q3
    ]).round(2)
    
    summary_stats.columns = ['count', 'mean', 'median', 'std', 'min', 'max', 'q25', 'q75']
    summary_stats = summary_stats.reset_index()
    
    # Add category descriptions
    summary_stats['category_description'] = summary_stats['Category'].map(category_descriptions)
    
    summary_stats['attribute'] = var_name

    # Save detailed results
    # summary_stats.to_csv('ar6_co2_price_publication_data.csv', index=False)
    
    return summary_stats


all_results = []

# 1. CO2 Price Analysis
result_co2 = analyze_one_variable(prepare_one_variable_long(scenarios_df_clean, 'Price|Carbon'),'CO2 price')
all_results.append(result_co2)
print(f"✅ CO2 price: {len(result_co2)} records")

# 2. Electricity Growth (indexed to 2020)
result_elec_growth = analyze_one_variable(prepare_idx_variable('Final Energy|Electricity', 2020),'Electricity growth relative to 2020')
all_results.append(result_elec_growth)
print(f"✅ Electricity growth: {len(result_elec_growth)} records")

# 3. Hydrogen vs Electricity Ratio
result_hydrogen = analyze_one_variable(prepare_ratio_variable('Final Energy|Hydrogen','Final Energy|Electricity'),'Hydrogen as a share of electricity')
all_results.append(result_hydrogen)
print(f"✅ Hydrogen share: {len(result_hydrogen)} records")

# 4. Industry Electricity Share
result_industry = analyze_one_variable(prepare_ratio_variable('Final Energy|Industry|Electricity','Final Energy|Electricity'),'Industry electricity share')
all_results.append(result_industry)
print(f"✅ Industry share: {len(result_industry)} records")

# 5. Residential & Commercial Share
result_rescom = analyze_one_variable(prepare_ratio_variable('Final Energy|Residential and Commercial|Electricity','Final Energy|Electricity'),'Residential and commercial electricity share')
all_results.append(result_rescom)
print(f"✅ Residential & commercial share: {len(result_rescom)} records")

# 6. Transportation Electrification
result_transport = analyze_one_variable(prepare_ratio_variable('Final Energy|Transportation|Electricity','Final Energy|Electricity'),'Transportation electricity share')
all_results.append(result_transport)
print(f"✅ Transportation share: {len(result_transport)} records")

# 7. Gas Price Analysis
result_gas = analyze_one_variable(prepare_idx_variable('Price|Secondary Energy|Gases|Natural Gas',2020),'Gas price')
all_results.append(result_gas)
print(f"✅ Gas price: {len(result_gas)} records")

# 8. Oil Price Analysis
result_oil = analyze_one_variable(prepare_idx_variable('Price|Secondary Energy|Liquids|Oil',2020),'Oil price')
all_results.append(result_oil)
print(f"✅ Oil price: {len(result_oil)} records")

# 9. Coal Price Analysis
result_coal = analyze_one_variable(prepare_idx_variable('Price|Secondary Energy|Solids|Coal',2020),'Coal price')
all_results.append(result_coal)
print(f"✅ Coal price: {len(result_coal)} records")

# 10. Biomass Price Analysis
result_biomass = analyze_one_variable(prepare_idx_variable('Price|Secondary Energy|Solids|Biomass',2020),'Biomass price')
all_results.append(result_biomass)
print(f"✅ Biomass price: {len(result_biomass)} records")


# Combine all DataFrames
combined_df = pd.concat(all_results, ignore_index=True)

# Save to CSV
output_file = 'ar6_r10_scenario_drivers.csv'
combined_df.to_csv(output_file, index=False)

print(f"\n🎯 Combined dataset saved: {output_file}")
print(f"📊 Total records: {len(combined_df)}")
print(f"📈 Attributes: {combined_df['attribute'].nunique()}")
print(f"🌍 Regions: {combined_df['Region'].nunique()}")
print(f"🎭 Categories: {combined_df['Category'].nunique()}")
print(f"📅 Years: {sorted(combined_df['Year'].unique())}")

# Show summary by attribute
print(f"\n📋 Summary by Attribute:")
summary_by_attr = combined_df.groupby('attribute').size().reset_index(name='record_count')
for _, row in summary_by_attr.iterrows():
    print(f"  {row['attribute']}: {row['record_count']} records")

display(combined_df.head(10))



In [ ]:
# with the raw dataframe

import pandas as pd

def analyze_one_variable(var_data,var_name):
    """Analyze distribution of a variable across scenarios and regions"""
    
    category_descriptions = {
        'C1': 'Limit warming to 1.5°C (>50%) with no or limited overshoot',
        'C2': 'Limit warming to 1.5°C (>67%) with high overshoot',
        'C3': 'Limit warming to 2°C (>67%) with higher action post-2030', 
        'C4': 'Limit warming to 2°C (>50%) with immediate action',
        'C7': 'Likely above 3°C warming with limited mitigation'
    }
    
    # Calculate summary statistics for each category-region-year combination
    summary_stats = var_data.groupby(['Category', 'Region', 'Year'])['Value'].agg([
        'count', 'mean', 'median', 'std', 'min', 'max',
        lambda x: x.quantile(0.25),  # Q1
        lambda x: x.quantile(0.75)   # Q3
    ]).round(2)
    
    summary_stats.columns = ['count', 'mean', 'median', 'std', 'min', 'max', 'q25', 'q75']
    summary_stats = summary_stats.reset_index()
    
    # Add category descriptions
    summary_stats['category_description'] = summary_stats['Category'].map(category_descriptions)
    
    summary_stats['attribute'] = var_name

    # Save detailed results
    # summary_stats.to_csv('ar6_co2_price_publication_data.csv', index=False)
    
    return summary_stats

def prepare_one_variable_long(scenarios_df, variable_name):
    """Prepare data for distribution analysis"""
        
    # Filter scenarios data for variable == variable_name, case insensitive
    var_data = scenarios_df[scenarios_df['Variable'].str.lower() == variable_name.lower()].copy()
    

    # Identify columns that look like years (e.g., '2000', '2010', etc.)
    year_cols = [col for col in var_data.columns if re.match(r'^\d{4}$', str(col))]

    # Melt the data to long format
    id_cols = ['Model', 'Scenario', 'Region', 'Variable', 'Unit']
    melted_data = var_data.melt(
        id_vars=id_cols,
        value_vars=year_cols,
        var_name='Year',
        value_name='Value'
    )
    
    # Convert Year to integer and filter out NaN values
    melted_data['Year'] = melted_data['Year'].astype(int)
    melted_data = melted_data.dropna(subset=['Value'])
    
    # Merge with metadata to get climate categories
    analysis_data = melted_data.merge(
        metadata_df[['Model', 'Scenario', 'Category', 'Category_name']], 
        on=['Model', 'Scenario'], 
        how='left'
    )
    
    # Remove rows without category information
    analysis_data = analysis_data.dropna(subset=['Category'])
    
    # Filter for available categories only
    selected_categories = ['C1', 'C2', 'C3', 'C4', 'C7']
    analysis_data = analysis_data[analysis_data['Category'].isin(selected_categories)]

    selected_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]
    analysis_data = analysis_data[analysis_data['Year'].isin(selected_years)]


    return analysis_data

all_results = []

# 1. CO2 Price Analysis
result_co2 = analyze_one_variable(prepare_one_variable_long(scenarios_df, 'Price|Carbon'),'CO2 price')
all_results.append(result_co2)
print(f"✅ CO2 price: {len(result_co2)} records")

# 7. Gas Price Analysis
result_gas = analyze_one_variable(prepare_one_variable_long(scenarios_df, 'Price|Secondary Energy|Gases|Natural Gas'),'Gas price')
all_results.append(result_gas)
print(f"✅ Gas price: {len(result_gas)} records")

# 8. Oil Price Analysis
result_oil = analyze_one_variable(prepare_one_variable_long(scenarios_df, 'Price|Secondary Energy|Liquids|Oil'),'Oil price')
all_results.append(result_oil)
print(f"✅ Oil price: {len(result_oil)} records")

# 9. Coal Price Analysis
result_coal = analyze_one_variable(prepare_one_variable_long(scenarios_df, 'Price|Secondary Energy|Solids|Coal'),'Coal price')
all_results.append(result_coal)
print(f"✅ Coal price: {len(result_coal)} records")

# 10. Biomass Price Analysis
result_biomass = analyze_one_variable(prepare_one_variable_long(scenarios_df, 'Price|Secondary Energy|Solids|Biomass'),'Biomass price')
all_results.append(result_biomass)
print(f"✅ Biomass price: {len(result_biomass)} records")


# Combine all DataFrames
combined_df = pd.concat(all_results, ignore_index=True)

# Save to CSV
output_file = 'ar6_r10_scenario_drivers_raw.csv'
combined_df.to_csv(output_file, index=False)

print(f"\n🎯 Combined dataset saved: {output_file}")
print(f"📊 Total records: {len(combined_df)}")
print(f"📈 Attributes: {combined_df['attribute'].nunique()}")
print(f"🌍 Regions: {combined_df['Region'].nunique()}")
print(f"🎭 Categories: {combined_df['Category'].nunique()}")
print(f"📅 Years: {sorted(combined_df['Year'].unique())}")

# Show summary by attribute
print(f"\n📋 Summary by Attribute:")
summary_by_attr = combined_df.groupby('attribute').size().reset_index(name='record_count')
for _, row in summary_by_attr.iterrows():
    print(f"  {row['attribute']}: {row['record_count']} records")




In [19]:
# 🎨 PUBLICATION-QUALITY VISUALIZATION PANEL FOR VERVESTACKS WEBSITE
# "The Energy Transition Landscape" - Multi-Panel Dashboard

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd

def create_energy_transition_landscape(combined_df):
    """
    Create "The Energy Transition Landscape" - Multi-Panel Dashboard
    Option 1: Sankey + Small Multiples + Regional Heatmap
    """
    
    print("🎨 Creating 'The Energy Transition Landscape' dashboard...")
    
    # Define professional color palette for climate categories
    climate_colors = {
        'C1': '#1f4e79',  # Deep blue - most ambitious
        'C2': '#2e75b6',  # Medium blue  
        'C3': '#43aa8b',  # Teal - moderate
        'C4': '#f8961e',  # Orange - delayed action
        'C7': '#f94144'   # Red - limited mitigation
    }
    
    # ========================================
    # PANEL A: SCENARIO BREADTH OVERVIEW (SANKEY)
    # ========================================
    
    print("📊 Creating Panel A: Scenario Breadth Overview...")
    
    # Calculate actual numbers from the data
    total_scenarios = len(combined_df[combined_df['Year'] == 2050])
    total_regions = combined_df['Region'].nunique()
    total_variables = combined_df['attribute'].nunique()
    total_categories = combined_df['Category'].nunique()
    
    # Create Sankey diagram
    fig_sankey = go.Figure(data=[go.Sankey(
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color = "black", width = 0.5),
            label = [
                "IPCC AR6 WG3\nDatabase",
                "Chapter 3\nVetted Scenarios", 
                f"{total_categories} Climate\nCategories",
                f"{total_variables} Key Energy\nVariables",
                f"190+ VerveStacks\nCountry Models"
            ],
            color = ["#34495e", "#3498db", "#2ecc71", "#f39c12", "#e74c3c"]
        ),
        link = dict(
            source = [0, 1, 2, 3],
            target = [1, 2, 3, 4],
            value = [1000, 400, 400, 190],
            color = ["rgba(52,73,94,0.3)", "rgba(52,152,219,0.3)", "rgba(46,204,113,0.3)", "rgba(243,156,18,0.3)"]
        )
    )])
    
    fig_sankey.update_layout(
        title=dict(
            text="<b>Panel A: Scenario Breadth & Quality Assurance</b><br>" +
                 "<sub>Rigorous filtering from global database to country-specific models</sub>",
            x=0.5,
            font=dict(size=16, color='#2c3e50')
        ),
        font=dict(family="Source Sans Pro, Arial", size=12),
        paper_bgcolor='white',
        width=800,
        height=400,
        margin=dict(l=20, r=20, t=80, b=20)
    )
    
    # ========================================
    # PANEL B: MULTI-VARIABLE SMALL MULTIPLES
    # ========================================
    
    print("📈 Creating Panel B: Multi-Variable Trajectory Grid...")
    
    # Select 6 key variables for small multiples
    key_variables = [
        'CO2 price',
        'Electricity growth relative to 2020',
        'Transportation electricity share', 
        'Industry electricity share',
        'Hydrogen as a share of electricity',
        'Gas price'
    ]
    
    # Filter available variables
    available_vars = [var for var in key_variables if var in combined_df['attribute'].unique()][:6]
    
    # Create 2x3 subplot grid
    fig_multiples = make_subplots(
        rows=2, cols=3,
        subplot_titles=[var.replace(' relative to 2020', ' Growth').replace(' electricity share', ' Electrification').replace(' as a share of electricity', '/Electricity Ratio') for var in available_vars],
        vertical_spacing=0.12,
        horizontal_spacing=0.08
    )
    
    # Add traces for each variable
    for i, variable in enumerate(available_vars):
        row = (i // 3) + 1
        col = (i % 3) + 1
        
        var_data = combined_df[combined_df['attribute'] == variable]
        
        for category in ['C1', 'C2', 'C3', 'C4', 'C7']:
            cat_data = var_data[var_data['Category'] == category]
            
            if len(cat_data) > 0:
                # Add median line with ribbon
                fig_multiples.add_trace(
                    go.Scatter(
                        x=cat_data['Year'],
                        y=cat_data['median'],
                        mode='lines',
                        name=category if i == 0 else None,  # Only show legend for first subplot
                        showlegend=i == 0,
                        line=dict(color=climate_colors[category], width=2),
                        hovertemplate=f'<b>{category}</b><br>Year: %{{x}}<br>Median: %{{y:.1f}}<extra></extra>'
                    ),
                    row=row, col=col
                )
                
                # Add uncertainty ribbon (Q25-Q75)
                fig_multiples.add_trace(
                    go.Scatter(
                        x=list(cat_data['Year']) + list(cat_data['Year'][::-1]),
                        y=list(cat_data['q75']) + list(cat_data['q25'][::-1]),
                        fill='toself',
                        fillcolor=climate_colors[category].replace('rgb', 'rgba').replace(')', ', 0.15)'),
                        line=dict(color='rgba(255,255,255,0)'),
                        showlegend=False,
                        hoverinfo='skip'
                    ),
                    row=row, col=col
                )
    
    fig_multiples.update_layout(
        title=dict(
            text="<b>Panel B: Energy System Driver Trajectories by Climate Ambition</b><br>" +
                 "<sub>Median pathways with Q25-Q75 uncertainty bands • 2020-2050 evolution</sub>",
            x=0.5,
            font=dict(size=16, color='#2c3e50')
        ),
        font=dict(family="Source Sans Pro, Arial", size=10),
        paper_bgcolor='white',
        plot_bgcolor='white',
        width=1200,
        height=600,
        margin=dict(l=60, r=60, t=100, b=60),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.15,
            xanchor="center",
            x=0.5,
            title="Climate Categories"
        )
    )
    
    # Update axes for better readability
    for i in range(1, 7):
        row = ((i-1) // 3) + 1
        col = ((i-1) % 3) + 1
        fig_multiples.update_xaxes(title_text="Year", row=row, col=col)
        fig_multiples.update_yaxes(title_text="Value", row=row, col=col)
    
    # ========================================
    # PANEL C: REGIONAL VARIATION HEATMAP
    # ========================================
    
    print("🌍 Creating Panel C: Regional Variation Matrix...")
    
    # Create heatmap data for 2050 values
    data_2050 = combined_df[combined_df['Year'] == 2050].copy()
    
    # Select top 4 variables and regions for cleaner visualization
    top_vars = ['CO2 price', 'Electricity growth relative to 2020', 'Transportation electricity share', 'Industry electricity share']
    available_top_vars = [var for var in top_vars if var in data_2050['attribute'].unique()]
    
    # Create pivot table
    heatmap_data = data_2050.pivot_table(
        values='median',
        index='Region', 
        columns='attribute',
        aggfunc='mean'
    )
    
    # Filter to available variables
    heatmap_data = heatmap_data[available_top_vars] if available_top_vars else heatmap_data.iloc[:, :4]
    
    fig_heatmap = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=[col.replace(' relative to 2020', ' Growth').replace(' electricity share', ' Elec.') for col in heatmap_data.columns],
        y=[region.replace('R10', '') for region in heatmap_data.index],
        colorscale='RdYlBu_r',
        colorbar=dict(title="Median Value<br>2050"),
        hoverongaps=False,
        hovertemplate='<b>%{y}</b><br>%{x}<br>Value: %{z:.1f}<extra></extra>'
    ))
    
    # Add text annotations
    annotations = []
    for i, region in enumerate(heatmap_data.index):
        for j, variable in enumerate(heatmap_data.columns):
            value = heatmap_data.iloc[i, j]
            if not np.isnan(value):
                annotations.append(
                    dict(
                        x=j, y=i,
                        text=f'{value:.1f}',
                        showarrow=False,
                        font=dict(color='white' if abs(value - heatmap_data.values.mean()) > heatmap_data.values.std() else 'black', size=10)
                    )
                )
    
    fig_heatmap.update_layout(
        title=dict(
            text="<b>Panel C: Regional Variation in Energy Drivers (2050)</b><br>" +
                 "<sub>Median values across all climate scenarios • Darker = higher values</sub>",
            x=0.5,
            font=dict(size=16, color='#2c3e50')
        ),
        font=dict(family="Source Sans Pro, Arial", size=12),
        paper_bgcolor='white',
        width=800,
        height=500,
        margin=dict(l=80, r=80, t=100, b=60),
        annotations=annotations
    )
    
    return fig_sankey, fig_multiples, fig_heatmap

# Create the Energy Transition Landscape dashboard
print("🚀 Generating 'The Energy Transition Landscape' dashboard...")
sankey_fig, multiples_fig, heatmap_fig = create_energy_transition_landscape(combined_df)

print("✅ Dashboard created!")
print("\n📋 Dashboard Components:")
print("🔄 Panel A: Scenario Breadth Overview (Sankey diagram)")
print("📈 Panel B: Multi-Variable Trajectories (2x3 small multiples)")
print("🌍 Panel C: Regional Variation Matrix (Heatmap)")

# Display the visualizations
print("\n🎨 Displaying Energy Transition Landscape dashboard...")
sankey_fig.show()
multiples_fig.show()
heatmap_fig.show()

# Save as SVG for website use
print("\n💾 Saving SVG files for website integration...")
sankey_fig.write_image("vervestacks_scenario_breadth.svg", width=800, height=400)
multiples_fig.write_image("vervestacks_trajectory_grid.svg", width=1200, height=600)
heatmap_fig.write_image("vervestacks_regional_matrix.svg", width=800, height=500)

print("✅ SVG files saved for website use!")
print("📁 Files:")
print("  • vervestacks_scenario_breadth.svg (Panel A)")
print("  • vervestacks_trajectory_grid.svg (Panel B - Main hero visual)")
print("  • vervestacks_regional_matrix.svg (Panel C)")

# Summary statistics
print(f"\n📊 Dashboard Statistics:")
print(f"📈 Variables visualized: {combined_df['attribute'].nunique()}")
print(f"🌍 Regions covered: {combined_df['Region'].nunique()}")
print(f"🎭 Climate categories: {combined_df['Category'].nunique()}")
print(f"📅 Time span: {combined_df['Year'].min()}-{combined_df['Year'].max()}")
print(f"🔢 Total data points: {len(combined_df):,}")

print("\n🎯 Perfect for VerveStacks website - shows scientific rigor and global scope!")


🚀 Generating 'The Energy Transition Landscape' dashboard...
🎨 Creating 'The Energy Transition Landscape' dashboard...
📊 Creating Panel A: Scenario Breadth Overview...
📈 Creating Panel B: Multi-Variable Trajectory Grid...
🌍 Creating Panel C: Regional Variation Matrix...
✅ Dashboard created!

📋 Dashboard Components:
🔄 Panel A: Scenario Breadth Overview (Sankey diagram)
📈 Panel B: Multi-Variable Trajectories (2x3 small multiples)
🌍 Panel C: Regional Variation Matrix (Heatmap)

🎨 Displaying Energy Transition Landscape dashboard...



💾 Saving SVG files for website integration...
✅ SVG files saved for website use!
📁 Files:
  • vervestacks_scenario_breadth.svg (Panel A)
  • vervestacks_trajectory_grid.svg (Panel B - Main hero visual)
  • vervestacks_regional_matrix.svg (Panel C)

📊 Dashboard Statistics:
📈 Variables visualized: 10
🌍 Regions covered: 11
🎭 Climate categories: 5
📅 Time span: 2020-2050
🔢 Total data points: 3,815

🎯 Perfect for VerveStacks website - shows scientific rigor and global scope!


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

def create_co2_visualizations(combined_df):
    """Create three visualization options for CO2 price data"""
    
    # Filter for CO2 price data only
    co2_data = combined_df[combined_df['attribute'] == 'CO2 price'].copy()
    
    print(f"📊 Creating visualizations for {len(co2_data)} CO2 price records...")
    
    # ========================================
    # 1. INTERACTIVE TRAJECTORY PLOT
    # ========================================
    
    fig1 = go.Figure()
    
    # Color palette for categories
    colors = {
        'C1': '#1f77b4',  # Blue
        'C2': '#ff7f0e',  # Orange  
        'C3': '#2ca02c',  # Green
        'C4': '#d62728',  # Red
        'C7': '#9467bd'   # Purple
    }
    
    for category in ['C1', 'C2', 'C3', 'C4', 'C7']:
        cat_data = co2_data[co2_data['Category'] == category]
        
        if len(cat_data) > 0:
            # Get category description
            cat_desc = cat_data['category_description'].iloc[0]
            
            # Add median line
            fig1.add_trace(go.Scatter(
                x=cat_data['Year'],
                y=cat_data['median'],
                mode='lines+markers',
                name=f'{category}: {cat_desc[:50]}...',
                line=dict(color=colors[category], width=3),
                marker=dict(size=8),
                hovertemplate=f'<b>{category}</b><br>' +
                             'Year: %{x}<br>' +
                             'Median: $%{y:.0f}/tCO2<br>' +
                             '<extra></extra>'
            ))
            
            # Add Q25 and Q75 lines (thin lines instead of filled areas)
            fig1.add_trace(go.Scatter(
                x=cat_data['Year'],
                y=cat_data['q25'],
                mode='lines',
                name=f'{category} Q25',
                line=dict(color=colors[category], width=1, dash='dot'),
                showlegend=False,
                hovertemplate=f'<b>{category} Q25</b><br>' +
                             'Year: %{x}<br>' +
                             'Q25: $%{y:.0f}/tCO2<br>' +
                             '<extra></extra>'
            ))
            
            fig1.add_trace(go.Scatter(
                x=cat_data['Year'],
                y=cat_data['q75'],
                mode='lines',
                name=f'{category} Q75',
                line=dict(color=colors[category], width=1, dash='dot'),
                showlegend=False,
                hovertemplate=f'<b>{category} Q75</b><br>' +
                             'Year: %{x}<br>' +
                             'Q75: $%{y:.0f}/tCO2<br>' +
                             '<extra></extra>'
            ))
    
    fig1.update_layout(
        title='CO2 Price Trajectories by Climate Category<br><sub>Median values with Q25-Q75 uncertainty bands</sub>',
        xaxis_title='Year',
        yaxis_title='CO2 Price (USD/tCO2)',
        yaxis_type='log',
        width=1000,
        height=600,
        template='plotly_white',
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
    )
    
    # ========================================
    # 2. RIDGE PLOTS (DENSITY EVOLUTION)
    # ========================================
    
    # Create ridge plot using subplots
    years = sorted(co2_data['Year'].unique())
    fig2 = make_subplots(
        rows=len(years), cols=1,
        subplot_titles=[f'Year {year}' for year in years],
        vertical_spacing=0.02,
        shared_xaxes=True
    )
    
    for i, year in enumerate(years):
        year_data = co2_data[co2_data['Year'] == year]
        
        for category in ['C1', 'C2', 'C3', 'C4', 'C7']:
            cat_year_data = year_data[year_data['Category'] == category]
            
            if len(cat_year_data) > 0:
                # Create density curve using median ± std approximation
                median = cat_year_data['median'].iloc[0]
                std = cat_year_data['std'].iloc[0]
                q25 = cat_year_data['q25'].iloc[0]
                q75 = cat_year_data['q75'].iloc[0]
                
                # Generate points for density curve
                x_points = np.linspace(max(0, median - 3*std), median + 3*std, 100)
                # Simple gaussian approximation
                y_points = np.exp(-0.5 * ((x_points - median) / (std + 1e-6))**2)
                y_points = y_points / np.max(y_points) * 0.8  # Normalize height
                
                fig2.add_trace(go.Scatter(
                    x=x_points,
                    y=y_points + i,  # Offset by row
                    mode='lines',
                    fill='tonexty' if category == 'C1' else 'tozeroy',
                    name=category if i == 0 else None,  # Only show legend for first year
                    showlegend=i == 0,
                    line=dict(color=colors[category], width=2),
                    fillcolor=colors[category].replace('rgb', 'rgba').replace(')', ', 0.3)'),
                    hovertemplate=f'<b>{category} - {year}</b><br>' +
                                 f'Median: ${median:.0f}/tCO2<br>' +
                                 f'Q25-Q75: ${q25:.0f}-${q75:.0f}<br>' +
                                 '<extra></extra>'
                ), row=i+1, col=1)
    
    fig2.update_layout(
        title='CO2 Price Distribution Evolution by Climate Category<br><sub>Ridge plots showing density evolution over time</sub>',
        xaxis_title='CO2 Price (USD/tCO2)',
        width=1000,
        height=200 * len(years),
        template='plotly_white',
        showlegend=True
    )
    
    # Remove y-axis labels for cleaner look
    for i in range(len(years)):
        fig2.update_yaxes(showticklabels=False, row=i+1, col=1)
    
    # ========================================
    # 3. HEATMAP OF MEDIANS
    # ========================================
    
    # Pivot data for heatmap
    heatmap_data = co2_data.pivot_table(
        values='median', 
        index='Category', 
        columns='Year', 
        aggfunc='mean'  # In case of duplicates
    )
    
    # Create annotations for the heatmap
    annotations = []
    for i, category in enumerate(heatmap_data.index):
        for j, year in enumerate(heatmap_data.columns):
            value = heatmap_data.iloc[i, j]
            if not np.isnan(value):
                annotations.append(
                    dict(
                        x=year, y=category,
                        text=f'${value:.0f}',
                        showarrow=False,
                        font=dict(color='white' if value > heatmap_data.values.max()/2 else 'black')
                    )
                )
    
    fig3 = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='Viridis',
        colorbar=dict(title='CO2 Price<br>(USD/tCO2)'),
        hoverongaps=False,
        hovertemplate='Year: %{x}<br>Category: %{y}<br>Median Price: $%{z:.0f}/tCO2<extra></extra>'
    ))
    
    fig3.update_layout(
        title='CO2 Price Heatmap: Median Values by Category and Year<br><sub>Quick pattern recognition across time and climate ambition</sub>',
        xaxis_title='Year',
        yaxis_title='Climate Category',
        width=800,
        height=400,
        template='plotly_white',
        annotations=annotations
    )
    
    return fig1, fig2, fig3

# Create the visualizations
print("🎨 Creating CO2 price visualizations...")
trajectory_fig, ridge_fig, heatmap_fig = create_co2_visualizations(combined_df)

print("✅ All visualizations created!")
print("\n📊 Visualization Options:")
print("1. Interactive Trajectory Plot - Explore median trends with uncertainty")
print("2. Ridge Plots - See distribution evolution over time") 
print("3. Heatmap - Quick pattern recognition across categories and years")

# Display the plots
trajectory_fig.show()
ridge_fig.show() 
heatmap_fig.show()
